In [1]:
from sentence_transformers import SentenceTransformer, util
import fitz
import os

# 初始化嵌入模型
embed_model = SentenceTransformer("BAAI/bge-m3")




local_chunk_set = []


def read_pdf(pdf_path):
    """讀取 PDF 檔案並依據文本頁數返回其內容"""
    page_content= []
    doc = fitz.open(pdf_path)
    for page_num, page in enumerate(doc):
        text = page.get_text()
        page_content.append({'page':page_num+1,'content':text})
    doc.close()
    return page_content


def chunk_text(pages, chunk_size):
    """將文本拆分為指定大小的 chunks，允許 chunk 跨越頁數，並標記 chunk 涉及的頁數範圍"""
    chunks = []
    current_chunk = []
    current_pages = set()
    current_length = 0
    max_token_length = chunk_size  

    for page in pages:
        text = page['content']
        text_tokens = embed_model.tokenizer.tokenize(text)

        while text_tokens:
            space_left = max_token_length - current_length

            # 如果當前 chunk 還有空間
            if space_left > 0:
                tokens_to_add = text_tokens[:space_left]
                text_tokens = text_tokens[space_left:]

                current_chunk.extend(tokens_to_add)
                current_pages.add(page['page'])
                current_length += len(tokens_to_add)

            # 當 chunk 滿了，就存入 chunks，並重置變數
            if current_length >= max_token_length or (current_length > 0 and len(text_tokens) > 0):
                chunks.append({
                    'pages': sorted(current_pages),
                    'content': embed_model.tokenizer.convert_tokens_to_string(current_chunk)
                })
                current_chunk = []
                current_pages = set()
                current_length = 0

    # 處理最後一個未滿的 chunk
    if current_length > 0:
        chunks.append({
            'pages': sorted(current_pages),
            'content': embed_model.tokenizer.convert_tokens_to_string(current_chunk)
        })

    return chunks


c:\Users\hank\AppData\Local\Programs\Python\Python38\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [3]:
folder_path = '../pdf'
pdf_files = [f for f in os.listdir(folder_path) if f.lower().endswith('.pdf')]


原始的chunk

In [ ]:
import os
import faiss  
import numpy as np
import json
from rouge_score import rouge_scorer
from sentence_transformers import SentenceTransformer
from mauve import compute_mauve
from langchain_community.llms.ollama import Ollama

def calculatge_mauve(reference, candidate):
    return compute_mauve(p_text=reference, q_text=candidate, device_id=0).mauve

def lcs_length(s1, s2):
    m, n = len(s1), len(s2)
    dp = [[0] * (n + 1) for _ in range(m + 1)]
    
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if s1[i - 1] == s2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1] + 1
            else:
                dp[i][j] = max(dp[i - 1][j], dp[i][j - 1])
    
    return dp[m][n]

def calculate_rouge_l(candidate, reference):
    lcs = lcs_length(candidate, reference)
    p = lcs / len(candidate) if candidate else 0
    r = lcs / len(reference) if reference else 0
    
    if p + r == 0:
        return 0.0
    
    f1 = (2 * p * r) / (p + r)
    return f1


def search_faiss(query_text, top_k=3):
    """搜尋最相近的前 top_k 筆資料"""
    query_embedding = embed_model.encode(query_text).astype(np.float32)
    query_embedding = np.expand_dims(query_embedding, axis=0)  # FAISS 需要的 2D 陣列

    # 查詢 FAISS
    _, indices = index.search(query_embedding, top_k)

    results = []
    for idx in indices[0]:
        if str(idx) in faiss_metadata:
            results.append(faiss_metadata[str(idx)])
    return results


llm = Ollama(model="jcai/llama-3-taiwan-8b-instruct:q4_k_m", temperature=0.2)

# 儲存 FAISS ID 對應的 metadata

# 設定批次大小，避免 OOM
result = []
chunk_sizes = [64, 128, 256, 512]  # 測試不同 chunk sizes
for size in chunk_sizes:
    EMBEDDING_DIM = 1024  
    index = faiss.IndexFlatL2(EMBEDDING_DIM)  # 建立 L2 距離索引
    data = []
    faiss_metadata = {}
    files_to_delete = ['faiss_metadata.json', 'faiss_index.idx']
    for file in files_to_delete:
        if os.path.exists(file):
            os.remove(file)
    for pdf_file in pdf_files:
        full_path = os.path.join(folder_path, pdf_file)
        content = read_pdf(full_path)
        chunks = chunk_text(content, size)
        for idx, chunk in enumerate(chunks):
            text = chunk["content"]
            # 計算 embedding 並存入 cache
            embedding = embed_model.encode(text).astype(np.float32)

            # 儲存 metadata
            data.append({
                "File_Name": pdf_file,
                "content": text,
                "Page_Num": ','.join(str(p) for p in chunk["pages"])
            })
            index.add(np.array([embedding]))  # 加入 FAISS
            faiss_metadata[len(faiss_metadata)] = data[-1]  # FAISS ID 對應 metadata
    # 儲存 FAISS 索引
    with open('../law_data.json', 'r', encoding='utf-8') as f:
        law_data = json.load(f)
    for idx, chunk in enumerate(law_data):
        text = chunk["law_text"]
        # 計算 embedding 並存入 cache
        embedding = embed_model.encode(text).astype(np.float32)

        # 儲存 metadata
        data.append({
            "File_Name": chunk["law_name"],
            "content": text,
            "Page_Num": ''
        })
        index.add(np.array([embedding]))  # 加入 FAISS
        faiss_metadata[len(faiss_metadata)] = data[-1]  # FAISS ID 對應 metadata
    faiss.write_index(index, "faiss_index.idx")
    with open("faiss_metadata.json", "w", encoding="utf-8") as f:
        json.dump(faiss_metadata, f, ensure_ascii=False, indent=4)
    print(f"已儲存 {len(faiss_metadata)} 筆資料到 FAISS")
    
    index = faiss.IndexFlatL2(EMBEDDING_DIM)
    index = faiss.read_index("faiss_index.idx")

    with open("faiss_metadata.json", "r", encoding="utf-8") as f:
        faiss_metadata = json.load(f)
    with open('pdf_questions.json', 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    win = 0
    loss = 0
    total_rouge = 0
    total_mauve= 0
    
    for each in data:
        res = search_faiss(each['question'])
        found = False
        
        # 整理 Context
        context = " ".join([metadata['content'] for metadata in res])
        
        # LLM Prompt
        prompt = f"""
        你是一個 AI 助理，專門根據官方文件回答資安相關問題。請根據提供的文件內容生成精確且簡潔的答案，不要加入額外資訊或推測性的內容，並確保答案與參考內容高度一致。

        **問題：** {each['question']}
        **參考內容：** {context}

        **答案（請直接從參考內容提取，確保準確性以及符合問題）：**
        """
        
        # 生成回答
        generated_answer = llm.invoke(prompt).strip()
        
        # 計算 ROUGE 分數
        rouge_score = calculate_rouge_l(each['answer'], generated_answer)
        mauve_score = calculatge_mauve(each['answer'], generated_answer)
        total_rouge += rouge_score
        total_mauve += mauve_score
        # 確認是否匹配文件名稱和頁碼
        for metadata in res:
            if metadata['File_Name'] == each['File_Name'] and str(each['Page_Num']) in metadata['Page_Num']:
                win += 1
                found = True
                break  # 若找到匹配則跳出
        if not found:
            loss += 1
    
    accuracy = win / (win + loss) if (win + loss) > 0 else 0
    result.append({
        'chunk_size': size,
        'total_chunk': len(faiss_metadata),
        'accuracy': accuracy,
        'rougeL': total_rouge / (win + loss),
        'mauve': total_mauve / (win + loss)
    })

for each in result:
    print(each)


已儲存 4940 筆資料到 FAISS


Featurizing q: 100%|██████████| 154/154 [00:02<00:00, 62.93it/s]


問題：如何在教育機構資安通報平台修改資安長的資料？
參考答案：請單位的資安連絡人登錄「教育機構資安通報平台」(https://info.cert.tanet.edu.tw)後，點選左方功能列之「修改資安長資料」功能項，輸入單位的資安長相關連絡資訊，點選「送出」以儲存資料。
生成答案：在教育機構資安通報平台修改資安長的資料，請單位的資安連絡人登錄「教育機構資安通報平台」(https://info.cert.tanet.edu.tw)後，點選左方功能列之「修改資安長資料」功能項。輸入單位的資安長相關連絡資訊（包含資安長姓名、資安長公務電話及資安長公務e-mail），點選「送出」以儲存資料。
ROUGE-L：0.803088803088803


Featurizing q: 100%|██████████| 36/36 [00:00<00:00, 64.51it/s]


問題：資安長的聯絡資訊需要包含哪些內容？
參考答案：資安長的聯絡資訊需要包含資安長姓名、資安長公務電話及資安長公務e-mail。
生成答案：資安長的聯絡資訊需要包含資安長姓名、資安長公務電話及資安長公務電子郵件。
ROUGE-L：0.8648648648648649


...

問題：如何新增或刪除通報平台的聯絡人？
參考答案：要新增或是刪除聯絡人，必須使用第一或第二聯絡人帳號，在帳號管理裡面做新增或是刪除後面聯絡人的帳號。
生成答案：要新增或刪除通報平台的聯絡人,必須使用第一或第二聯絡人帳號,在帳號管理裡面做新增或是刪除後面聯絡人的帳號。若聯絡人已不在職,可先申請重設密碼後請他人登入,並修改個人資料,以更換新的聯絡人;若在職,可直接請異動之聯絡人修改個人資料。
ROUGE-L：0.5609756097560976


Featurizing q: 100%|██████████| 47/47 [00:00<00:00, 58.69it/s]


問題：在EWA事件單中，如何處理確實事件？
參考答案：確實事件：經查證後為確實事件，須先「自行通報」，並填入自行通報的資安事件編號。
生成答案：在EWA事件單中，處理確實事件的方式為：經查證後須「自行通報」，並填入自行通報的資安事件編號。
ROUGE-L：0.7441860465116279


Featurizing q: 100%|██████████| 64/64 [00:01<00:00, 62.91it/s]


問題：在EWA事件單中，如何處理誤判事件？
參考答案：誤判：經查證後確認為誤判事件，並於「原因」欄位中，說明誤判原因。
生成答案：在EWA事件單中，處理誤判事件的方式是：經查證後確認為誤判，於處理時限內完成事件應變處理並結案。通報與填寫方式與正式事件單相同。
ROUGE-L：0.3333333333333333
已儲存 755 筆資料到 FAISS


計算 element chunk 的準確度

In [18]:

index = faiss.IndexFlatL2(EMBEDDING_DIM)
index = faiss.read_index("faiss_element_chunk_index.idx")
with open("faiss_element_chunk_metadata.json", "r", encoding="utf-8") as f:
    faiss_metadata = json.load(f)
with open('pdf_questions.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
total_bert = 0
total_rough=0
win = 0
loss = 0
for each in data : 
        # 整理 Context
    context = " ".join([metadata['content'] for metadata in res])
    # LLM Prompt
    prompt = f"""
    你是一個 AI 助理，專門根據官方文件回答資安相關問題。請根據提供的文件內容生成精確且簡潔的答案，不要加入額外資訊或推測性的內容，並確保答案與參考內容高度一致。

    **問題：** {each['question']}
    **參考內容：** {context}

    **答案（請直接從參考內容提取，確保準確性以及符合問題）：**
    """
    # 生成回答
    #generated_answer = llm.invoke(prompt).strip()
    # 計算 BLEU 和 ROUGE 分數
    rough_score = calculate_rouge_l(each['answer'], generated_answer)
    bert_score = calculatge_bert_score(each['answer'], generated_answer)    
    res = search_faiss(each['question'])
    found = False
    total_bert += bert_score
    total_rough += rough_score
    for metadata in res:
        if metadata['File_Name'] == each['File_Name'] and str(each['Page_Num']) in  metadata['Page_Num']:
            win += 1
            found = True
            break
    if not found:
        loss += 1

accuracy = win / (win + loss) if (win + loss) > 0 else 0
average_rough = total_rough / (win + loss) 
average_bert = total_bert / (win + loss)
print("Element Chunk")
print(f"Accuracy: {accuracy:.4f}")
print(f"total_chunk: {len(faiss_metadata)}")
print(f"Average bert score: {average_bert:.4f}")
print(f"Average ROUGE-L score: {average_rough:.4f}")


KeyboardInterrupt: 